### Text Selection Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
#path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Text Selection/Result/2-Text Selection Data Merge"

In [9]:
df_word = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx")
df_word = df_word.fillna("[]")
df_word = df_word.iloc[1000:1005,]
df_word

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,arka,69809,"['arka tarafta', 'arka tarafa', 'arka kapı', '...","['arka tarafta bir', 'bir arka kapı', 'arka ta...","['arka kapısı var mı', 'buranın arka kapısı va...","['buranın arka kapısı var mı', 'onlar sadece a...",['arka kapısında tv seti çalarken yakalamışlar...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...
1001,hoşuma,69776,"['hoşuma gitti', 'hepinizi kutsasın', 'bu hoşu...","['hiç hoşuma gitmedi', 'çok hoşuma gitti', 'bu...","['bu hiç hoşuma gitmedi', 'her şeyi bilen biri...","['bu iş hiç hoşuma gitmedi', 'bu benim de hoşu...","['bana şaplak atmana izin vermek hoşuma', 'şap...",['bana şaplak atmana izin vermek hoşuma gider'...,['tvde diğer ülkeleri falan görünce çok hoşuma...,['gece gelip çattı ve durum daha baştan hoşuma...,['büyük gece gelip çattı ve durum daha baştan ...
1002,üçüncü,69620,"['üçüncü bir', 'üçüncü kat', 've üçüncü', 'bu ...","['üçüncü ve dördüncü', 'bu hafta üçüncü', 'üçü...","['güney üçüncü sıra temiz', 'bu hafta üçüncü k...","['kuzey üçüncü sıra temiz sayım', 'üçüncü bir ...",['üçüncü mevkii yolcuları sağlık kontrolü için...,['üçüncü mevkii yolcuları sağlık kontrolü için...,['bütün üçüncü mevkii yolcuları sağlık kontrol...,['bütün üçüncü mevkii yolcuları sağlık kontrol...,['bütün üçüncü mevkii yolcuları sağlık kontrol...
1003,sırada,69586,"['sırada ne', 'o sırada', 'arada sırada', 'bu ...","['sırada ne var', 'sırada sen varsın', 'tam o ...","['peki sırada ne var', 'şimdi sırada ne var', ...","['top şu sırada sağ tarafta', 'arada sırada gü...","['bizi sırada ne tür bir macera', 'sırada ne t...","['sırada ne tür bir macera bekliyor acaba', 'b...",['bizi sırada ne tür bir macera bekliyor acaba...,['bu arkadaşın arkadaşa yapmayacağı şeyler lis...,['çünkü bu arkadaşın arkadaşa yapmayacağı şeyl...
1004,gidecek,69388,"['gidecek misin', 'yolunda gidecek', 'gidecek ...","['şey yolunda gidecek', 'çok hoşuna gidecek', ...","['geri getir olur mu', 'gidecek bir yerim yok'...","['gidecek başka bir yerim yok', 'gidecek bir y...","['burada bana eşlik etmen hoşuma gidecek', 'me...","['bu gece oraya bir adam uğradı mı', 'helikopt...",['adam gelecek ve sözde kayıp bavulunu alıp gi...,['gidecek birini rehin aldı ve ona kibar mı da...,['mahkemeye gidecek birini rehin aldı ve ona k...


In [10]:
df_twogram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx")
df_twogram = df_twogram.fillna("[]")
df_twogram = df_twogram.iloc[1000:1005,]
df_twogram

,twogram,frequency,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,ne biliyorsun,10048,"['hakkında ne biliyorsun', 'ne biliyorsun ki',...","['onun hakkında ne biliyorsun', 'hakkında ne b...","['bu konu hakkında ne biliyorsun', 'benim hakk...","['bu adam hakkında sen ne biliyorsun', 'v x ga...","['ne biliyorsun en son ne zaman davet', 'şu ha...","['ne biliyorsun en son ne zaman davet ettin', ...",[],['şu çocukları kurtaran adam hani şoför olan h...
1001,senin hatan,10010,"['senin hatan değil', 'bu senin hatan', 'senin...","['bu senin hatan değil', 'bu senin hatan değil...","['ama bu senin hatan değil', 'hayır bu senin h...","['bu senin hatan değil tamam mı', 'oğul olarak...","['oğul olarak senin hatan baba olarak benim', ...",['tabii ki senin hatan ben her şeyin farkınday...,['tüm bu çılgın gece tüm bu gece senin hatan'],['dinle beni bunun senin hatan olmadığına emin...
1002,hayır ama,9998,"['hayır ama bu', 'hayır ama bir', 'hayır ama b...","['hayır ama yine de', 'hayır ama ben seni', 'h...","['hayır ama ben seni tanıyorum', 'hayır ama ha...","['hayır ama halk üzerindeki etkisi tüm', 'hayı...","['hayır ama güzel çok çok iyi yüzbaşı', 'hayır...",['hayır ama çok iyi bir porno koleksiyonum var...,['hayır ama çok istediğin konuda sana bir şans...,['hayır ama beni kör eden ışığı gördüm ve sesi...
1003,daha kötü,9992,"['çok daha kötü', 'daha kötü bir', 'daha kötü ...","['daha kötü bir şey', 'daha kötü ne olabilir',...","['daha kötü ne olabilir ki', 'bundan daha kötü...","['bundan daha kötü ne olabilir ki', 'ne daha i...",['benim ama senin yüzün benimkinden daha kötü'...,['çürüyen benim ama senin yüzün benimkinden da...,['sonunda benden daha kötü bir şey buldunuz de...,['yaraya su döküp daha kötü yapabilir ya da ba...
1004,bunu biliyor,9992,"['bunu biliyor muydun', 'bunu biliyor musun', ...","['o da bunu biliyor', 'o bunu biliyor mu', 'se...","['ve o da bunu biliyor', 'bir baş belasısın bu...","['bunu biliyor ve korumak için savaşıyorlar', ...",['düzine kırmızı gül yollayacak bunu biliyor m...,['birer düzine kırmızı gül yollayacak bunu bil...,['size birer düzine kırmızı gül yollayacak bun...,['banka size birer düzine kırmızı gül yollayac...


In [11]:
df_threegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx")
df_threegram = df_threegram.fillna("[]")
df_threegram = df_threegram.iloc[1000:1005,]
df_threegram

,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,elimden geleni yaptım,3761,"['için elimden geleni yaptım', 'ben elimden ge...","['elimden geleni yaptım değil mi', 'korumak iç...","['yardım etmek için elimden geleni yaptım', 'k...",['elimden geleni yaptım ama adam müthiş fantas...,['elimden geleni yaptım ama adam müthiş fantas...,['ama o an geldi ve ben elimden geleni yaptım'...,['nazik ve sabırlı davranarak sana ulaşmak içi...
1001,beni bir daha,3754,"['beni bir daha asla', 'beni bir daha arama', ...","['beni bir daha asla görmeyeceksin', 'beni bir...","['beni bir daha arama tamam mı', 'beni bir dah...",['bu geceden sonra beni bir daha görmeyeceksin...,['eski karım beni bir daha görmek istemediğini...,[],['ama beni bir daha vurursan dua et de ölmüş o...
1002,bu gerçekten çok,3753,"['bu gerçekten çok iyi', 'bu gerçekten çok güz...","['bu gerçekten çok iyi bir', 'bu gerçekten çok...","['bu gerçekten çok iyi bir fikir', 'bu gerçekt...","['bu gerçekten çok zor bir karar oldu', 'bu ge...",['bu gerçekten çok zor olmaya başladı biliyor ...,[],[]
1003,sadece bir dakika,3751,"['sadece bir dakika sürer', 'sadece bir dakika...","['bana sadece bir dakika ver', 'sadece bir dak...",['sadece bir dakika yeter hayatınızı tehlikeye...,['sadece bir dakika yeter hayatınızı tehlikeye...,['bana sadece bir dakika yeter hayatınızı tehl...,[],['bu sadece bir dakika sürecek sonra üç dakika...
1004,ne zaman bir,3751,"['son ne zaman bir', 'ne zaman bir şey', 'ne z...","['en son ne zaman bir', 'son ne zaman bir şeyl...","['en son ne zaman bir şeyler', 'en son ne zama...","['en son ne zaman bir şeyler yedin', 'son ne z...","['en son ne zaman bir kadınla beraber oldun', ...",['ne zaman bir işi ilk seferde doğru yaptık ki...,['artık ne zaman bir fırsat görsem boğa gibi ü...


In [12]:
df_fourgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx")
df_fourgram = df_fourgram.fillna("[]")
df_fourgram = df_fourgram.iloc[1000:1005,]
df_fourgram

,fourgram,frequency,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,hiç de iyi değil,851,"['bu hiç de iyi değil', 'için hiç de iyi değil...","['clem bu hiç de iyi değil', 'hiç de iyi değil...","['tadı hiç de iyi değil değil mi', 've onların...","['ve onların tadı hiç de iyi değil değil', 'on...",['ve onların tadı hiç de iyi değil değil mi'],[]
1001,çok ama çok üzgünüm,850,"['için çok ama çok üzgünüm', 'gerçekten çok am...","['olanlar için çok ama çok üzgünüm', 'isterim ...",['bilmeni isterim ki çok ama çok üzgünüm'],['önce bilmeni isterim ki çok ama çok üzgünüm'...,['ama önce bilmeni isterim ki çok ama çok üzgü...,['sebep olduğum bu korkunç sıkıntılar için çok...
1002,olur ne olmaz diye,849,"['ne olur ne olmaz diye', 'olur ne olmaz diye ...","['ama ne olur ne olmaz diye', 'ne olur ne olma...","['ne olur ne olmaz diye kuleye asker', 'buraya...",['şeyleri buraya saklamıştım ne olur ne olmaz ...,['bazı şeyleri buraya saklamıştım ne olur ne o...,[]
1003,benimle dalga geçiyor olmalısın,849,['benimle dalga geçiyor olmalısın dostum'],[],[],[],[],[]
1004,gereken bir şey daha,849,"['gereken bir şey daha var', 'bilmen gereken b...","['bilmen gereken bir şey daha var', 'yapmam ge...","['sana söylemem gereken bir şey daha var', 'am...","['ve sana vermem gereken bir şey daha var', 'g...",['yapmam gereken bir şey daha var sonra tamame...,['yapmam gereken bir şey daha var sonra tamame...


In [13]:
df_fivegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx")
df_fivegram = df_fivegram.fillna("[]")
df_fivegram = df_fivegram.iloc[1000:1005,]
df_fivegram

,fivegram,frequency,sixgram,sevengram,eightgram,ninegram,tengram
1000,ben de sana aynı şeyi,274,"['ben de sana aynı şeyi soracaktım', 'ben de s...",[],[],[],[]
1001,başka bir çıkış var mı,274,"['buradan başka bir çıkış var mı', 'bu başka b...","['bu başka bir çıkış var mı demek', 'başka bir...",['bu başka bir çıkış var mı demek oluyor'],[],[]
1002,jüri bir karara vardı mı,272,"['foreman jüri bir karara vardı mı', 'sözcü jü...","['bay foreman jüri bir karara vardı mı', 'sayı...",[],[],[]
1003,eden bir şey mi var,272,"['rahatsız eden bir şey mi var', 'eden bir şey...","['seni rahatsız eden bir şey mi var', 'sizi ra...","['seni rahatsız eden bir şey mi var oğlum', 's...",[],[]
1004,için yapabileceğim bir şey varsa,272,"['senin için yapabileceğim bir şey varsa', 'si...",['eğer senin için yapabileceğim bir şey varsa'...,['gitmeden önce senin için yapabileceğim bir ş...,['eğer gitmeden önce senin için yapabileceğim ...,['daha rahat etmeniz için yapabileceğim bir şe...


In [14]:
df_sixgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx")
df_sixgram = df_sixgram.fillna("[]")
df_sixgram = df_sixgram.iloc[1000:1005,]
df_sixgram

,sixgram,frequency,sevengram,eightgram,ninegram,tengram
1000,ama önce benim için bir şey,60,"['ama önce benim için bir şey yapman', 'ama ön...","['ama önce benim için bir şey yapman gerek', '...",[],[]
1001,söylemek istediğin bir şey mi vardı,60,['bana söylemek istediğin bir şey mi vardı'],['bana özellikle söylemek istediğin bir şey mi...,[],[]
1002,çöz kalbimi lütfen gel sal beni,60,['gel çöz kalbimi lütfen gel sal beni'],[],[],[]
1003,bana şaplak atmana izin vermek hoşuma,60,['bana şaplak atmana izin vermek hoşuma gider'],[],[],[]
1004,yürüyüşüne devam et ve hep kendine,60,['kendince yürüyüşüne devam et ve hep kendine'...,['kendince yürüyüşüne devam et ve hep kendine ...,['sadece kendince yürüyüşüne devam et ve hep k...,[]


In [15]:
df_sevengram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx")
df_sevengram = df_sevengram.fillna("[]")
df_sevengram = df_sevengram.iloc[1000:1005,]
df_sevengram

,sevengram,frequency,eightgram,ninegram,tengram
1000,en önemli şey daima buradaki insanlar olacak,35,['hayattaki en önemli şey daima buradaki insan...,['ama hayattaki en önemli şey daima buradaki i...,[]
1001,ismi gelen iki pisliği geri götürmeye geldim,35,['masama ismi gelen iki pisliği geri götürmeye...,[],[]
1002,iki adam hakkında sana bir şey diyeyim,35,['bu iki adam hakkında sana bir şey diyeyim'],[],[]
1003,dostum işinizin ehli olduğunuzu biliyor ve saygı,35,['bak dostum işinizin ehli olduğunuzu biliyor ...,['bak dostum işinizin ehli olduğunuzu biliyor ...,[]
1004,şüphe yok ki gemiyi ve bizim hayatlarımızı,35,['hiç şüphe yok ki gemiyi ve bizim hayatlarımı...,['hiç şüphe yok ki gemiyi ve bizim hayatlarımı...,[]


In [16]:
df_eightgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx")
df_eightgram = df_eightgram.fillna("[]")
df_eightgram = df_eightgram.iloc[1000:1005,]
df_eightgram

,eightgram,frequency,ninegram,tengram
1000,düşünmüştüm ki sabah uyandığımda kendimi daha ...,29,['bugün düşünmüştüm ki sabah uyandığımda kendi...,[]
1001,çıkış noktası hakkında hiçbir şey bilmiyorsun ...,29,['müziğin çıkış noktası hakkında hiçbir şey bi...,['hiphop müziğin çıkış noktası hakkında hiçbir...
1002,olduğun yerde dur ve ellerini görünür şekilde tut,29,['zaman olduğun yerde dur ve ellerini görünür ...,['o zaman olduğun yerde dur ve ellerini görünü...
1003,ömür boyu içeride kalmamızı sağlayacak kadar t...,29,['ömür boyu içeride kalmamızı sağlayacak kadar...,[]
1004,adamla aranızda kan bağı vardır zira yaptığın ...,29,['umarım adamla aranızda kan bağı vardır zira ...,[]


In [17]:
df_ninegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx")
df_ninegram = df_ninegram.fillna("[]")
df_ninegram = df_ninegram.iloc[1000:1005,]
df_ninegram

,ninegram,frequency,tengram
1000,gün devam ederken david bunun en mutlu günü ol...,21,['gün devam ederken david bunun en mutlu günü ...
1001,tek istediğim ülkeme dönmek ve buna param yüz ...,21,['tek istediğim ülkeme dönmek ve buna param yü...
1002,gidip masaya oturmanı ve oraya bir isim yazman...,21,['mutfağa gidip masaya oturmanı ve oraya bir i...
1003,hanımlar kapıyı kapatıp bizi bir süreliğine ya...,21,['müsaadenizle hanımlar kapıyı kapatıp bizi bi...
1004,pilotun her an üzerinde olması gereken iki şey...,21,['bir pilotun her an üzerinde olması gereken i...


#### Sending Data To Firestore

##### Photo Data

In [18]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Text Data

##### Word

In [18]:
df_word["twogram"] = df_word["twogram"].apply(eval)
df_word["threegram"] = df_word["threegram"].apply(eval)
df_word["fourgram"] = df_word["fourgram"].apply(eval)
df_word["fivegram"] = df_word["fivegram"].apply(eval)
df_word["sixgram"] = df_word["sixgram"].apply(eval)
df_word["sevengram"] = df_word["sevengram"].apply(eval)
df_word["eightgram"] = df_word["eightgram"].apply(eval)
df_word["ninegram"] = df_word["ninegram"].apply(eval)
df_word["tengram"] = df_word["tengram"].apply(eval)

In [19]:
word = df_word["word"].to_list()
frequency_list = df_word["frequency"].to_list()
twogram_list = df_word["twogram"].to_list()
threegram_list = df_word["threegram"].to_list()
fourgram_list = df_word["fourgram"].to_list()
fivegram_list = df_word["fivegram"].to_list()
sixgram_list = df_word["sixgram"].to_list()
sevengram_list = df_word["sevengram"].to_list()
eightgram_list = df_word["eightgram"].to_list()
ninegram_list = df_word["ninegram"].to_list()
tengram_list = df_word["tengram"].to_list()

In [20]:
firestore_word_data = zip(word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [21]:
#word_data_list = []
#for word,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
#    data = {f"{word}":{"ninegram":ninegram_list,"tengram":tengram_list}}
#    
#    word_data_list.append({f"{word}":data})

In [22]:
word_dict = {}
for word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
    dict_var = {f"{word}":{"photo":"","frequency":frequency_list,"twogram":twogram_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[""]}}
    word_dict.update(dict_var)     

In [23]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").set(word_dict)

update_time {
  seconds: 1670338092
  nanos: 705631000
}

In [32]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").add(word_dict)

##### Update

In [42]:
update_word = "hoşuma"

In [45]:
dict_update = {f"{update_word}.photo":"www.store4.com",f"{update_word}.other":["gitti",{"zıt":"gelmek"}]}

In [46]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").update(dict_update)

ValueError: Non-alphanum char in element with leading alpha: hoşuma

##### Twogram

In [20]:
df_twogram["threegram"] = df_twogram["threegram"].apply(eval)
df_twogram["fourgram"] = df_twogram["fourgram"].apply(eval)
df_twogram["fivegram"] = df_twogram["fivegram"].apply(eval)
df_twogram["sixgram"] = df_twogram["sixgram"].apply(eval)
df_twogram["sevengram"] = df_twogram["sevengram"].apply(eval)
df_twogram["eightgram"] = df_twogram["eightgram"].apply(eval)
df_twogram["ninegram"] = df_twogram["ninegram"].apply(eval)
df_twogram["tengram"] = df_twogram["tengram"].apply(eval)

In [21]:
#twogram = df_twogram["twogram"].values.tolist()

In [22]:
twogram = df_twogram["twogram"].to_list()
frequency_list = df_twogram["frequency"].to_list()
threegram_list = df_twogram["threegram"].to_list()
fourgram_list = df_twogram["fourgram"].to_list()
fivegram_list = df_twogram["fivegram"].to_list()
sixgram_list = df_twogram["sixgram"].to_list()
sevengram_list = df_twogram["sevengram"].to_list()
eightgram_list = df_twogram["eightgram"].to_list()
ninegram_list = df_twogram["ninegram"].to_list()
tengram_list = df_twogram["tengram"].to_list()

In [23]:
firestore_twogram_data = zip(twogram,frequency_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [24]:
twogram_dict = {}
for twogram,frequency_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_twogram_data:
    dict_var = {f"{twogram}":{"photo":"","frequency":frequency_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[""]}}
    twogram_dict.update(dict_var)    

In [26]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("twogram_data").document(f"details").set(twogram_dict)

update_time {
  seconds: 1670337679
  nanos: 498159000
}

##### Threegram

In [103]:
df_threegram

,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,elimden geleni yaptım,3761,"['için elimden geleni yaptım', 'ben elimden ge...","['elimden geleni yaptım değil mi', 'korumak iç...","['yardım etmek için elimden geleni yaptım', 'k...",['elimden geleni yaptım ama adam müthiş fantas...,['elimden geleni yaptım ama adam müthiş fantas...,['ama o an geldi ve ben elimden geleni yaptım'...,['nazik ve sabırlı davranarak sana ulaşmak içi...
1001,beni bir daha,3754,"['beni bir daha asla', 'beni bir daha arama', ...","['beni bir daha asla görmeyeceksin', 'beni bir...","['beni bir daha arama tamam mı', 'beni bir dah...",['bu geceden sonra beni bir daha görmeyeceksin...,['eski karım beni bir daha görmek istemediğini...,NaN,['ama beni bir daha vurursan dua et de ölmüş o...
1002,bu gerçekten çok,3753,"['bu gerçekten çok iyi', 'bu gerçekten çok güz...","['bu gerçekten çok iyi bir', 'bu gerçekten çok...","['bu gerçekten çok iyi bir fikir', 'bu gerçekt...","['bu gerçekten çok zor bir karar oldu', 'bu ge...",['bu gerçekten çok zor olmaya başladı biliyor ...,NaN,NaN
1003,sadece bir dakika,3751,"['sadece bir dakika sürer', 'sadece bir dakika...","['bana sadece bir dakika ver', 'sadece bir dak...",['sadece bir dakika yeter hayatınızı tehlikeye...,['sadece bir dakika yeter hayatınızı tehlikeye...,['bana sadece bir dakika yeter hayatınızı tehl...,NaN,['bu sadece bir dakika sürecek sonra üç dakika...
1004,ne zaman bir,3751,"['son ne zaman bir', 'ne zaman bir şey', 'ne z...","['en son ne zaman bir', 'son ne zaman bir şeyl...","['en son ne zaman bir şeyler', 'en son ne zama...","['en son ne zaman bir şeyler yedin', 'son ne z...","['en son ne zaman bir kadınla beraber oldun', ...",['ne zaman bir işi ilk seferde doğru yaptık ki...,['artık ne zaman bir fırsat görsem boğa gibi ü...


In [ ]:
df_threegram["fourgram"] = df_threegram["fourgram"].apply(eval)
df_threegram["fivegram"] = df_threegram["fivegram"].apply(eval)
df_threegram["sixgram"] = df_threegram["sixgram"].apply(eval)
df_threegram["sevengram"] = df_threegram["sevengram"].apply(eval)
df_threegram["eightgram"] = df_threegram["eightgram"].apply(eval)
df_threegram["ninegram"] = df_threegram["ninegram"].apply(eval)
df_threegram["tengram"] = df_threegram["tengram"].apply(eval)

In [105]:
threegram = df_threegram["threegram"].to_list()
frequency_list = df_threegram["frequency"].to_list()
fourgram_list = df_threegram["fourgram"].to_list()
fivegram_list = df_threegram["fivegram"].to_list()
sixgram_list = df_threegram["sixgram"].to_list()
sevengram_list = df_threegram["sevengram"].to_list()
eightgram_list = df_threegram["eightgram"].to_list()
ninegram_list = df_threegram["ninegram"].to_list()
tengram_list = df_threegram["tengram"].to_list()

In [106]:
firestore_threegram_data = zip(threegram,frequency_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [107]:
threegram_dict = {}
for threegram,frequency_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_threegram_data:
    dict_var = {f"{threegram}":{"photo":"","frequency":frequency_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[""]}}
    threegram_dict.update(dict_var)    

In [108]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("threegram_data").document(f"details").set(threegram_dict)

update_time {
  seconds: 1670322555
  nanos: 625965000
}

##### Fourgram

In [109]:
df_fourgram

,fourgram,frequency,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,hiç de iyi değil,851,"['bu hiç de iyi değil', 'için hiç de iyi değil...","['clem bu hiç de iyi değil', 'hiç de iyi değil...","['tadı hiç de iyi değil değil mi', 've onların...","['ve onların tadı hiç de iyi değil değil', 'on...",['ve onların tadı hiç de iyi değil değil mi'],NaN
1001,çok ama çok üzgünüm,850,"['için çok ama çok üzgünüm', 'gerçekten çok am...","['olanlar için çok ama çok üzgünüm', 'isterim ...",['bilmeni isterim ki çok ama çok üzgünüm'],['önce bilmeni isterim ki çok ama çok üzgünüm'...,['ama önce bilmeni isterim ki çok ama çok üzgü...,['sebep olduğum bu korkunç sıkıntılar için çok...
1002,olur ne olmaz diye,849,"['ne olur ne olmaz diye', 'olur ne olmaz diye ...","['ama ne olur ne olmaz diye', 'ne olur ne olma...","['ne olur ne olmaz diye kuleye asker', 'buraya...",['şeyleri buraya saklamıştım ne olur ne olmaz ...,['bazı şeyleri buraya saklamıştım ne olur ne o...,NaN
1003,benimle dalga geçiyor olmalısın,849,['benimle dalga geçiyor olmalısın dostum'],NaN,NaN,NaN,NaN,NaN
1004,gereken bir şey daha,849,"['gereken bir şey daha var', 'bilmen gereken b...","['bilmen gereken bir şey daha var', 'yapmam ge...","['sana söylemem gereken bir şey daha var', 'am...","['ve sana vermem gereken bir şey daha var', 'g...",['yapmam gereken bir şey daha var sonra tamame...,['yapmam gereken bir şey daha var sonra tamame...


In [ ]:
df_fourgram["fivegram"] = df_fourgram["fivegram"].apply(eval)
df_fourgram["sixgram"] = df_fourgram["sixgram"].apply(eval)
df_fourgram["sevengram"] = df_fourgram["sevengram"].apply(eval)
df_fourgram["eightgram"] = df_fourgram["eightgram"].apply(eval)
df_fourgram["ninegram"] = df_fourgram["ninegram"].apply(eval)
df_fourgram["tengram"] = df_fourgram["tengram"].apply(eval)

In [110]:
fourgram = df_fourgram["fourgram"].to_list()
frequency_list = df_fourgram["frequency"].to_list()
fivegram_list = df_fourgram["fivegram"].to_list()
sixgram_list = df_fourgram["sixgram"].to_list()
sevengram_list = df_fourgram["sevengram"].to_list()
eightgram_list = df_fourgram["eightgram"].to_list()
ninegram_list = df_fourgram["ninegram"].to_list()
tengram_list = df_fourgram["tengram"].to_list()

In [111]:
firestore_fourgram_data = zip(fourgram,frequency_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [112]:
fourgram_dict = {}
for fourgram,frequency_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fourgram_data:
    dict_var = {f"{fourgram}":{"photo":"","frequency":frequency_list,"fivegram":fivegram_list,"sixgram":sixgram_list,
    "sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[""]}}
    fourgram_dict.update(dict_var)    

In [114]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fourgram_data").document(f"details").set(fourgram_dict)

update_time {
  seconds: 1670322912
  nanos: 86241000
}

##### Fivegram

In [115]:
df_fivegram

,fivegram,frequency,sixgram,sevengram,eightgram,ninegram,tengram
1000,ben de sana aynı şeyi,274,"['ben de sana aynı şeyi soracaktım', 'ben de s...",NaN,NaN,NaN,NaN
1001,başka bir çıkış var mı,274,"['buradan başka bir çıkış var mı', 'bu başka b...","['bu başka bir çıkış var mı demek', 'başka bir...",['bu başka bir çıkış var mı demek oluyor'],NaN,NaN
1002,jüri bir karara vardı mı,272,"['foreman jüri bir karara vardı mı', 'sözcü jü...","['bay foreman jüri bir karara vardı mı', 'sayı...",NaN,NaN,NaN
1003,eden bir şey mi var,272,"['rahatsız eden bir şey mi var', 'eden bir şey...","['seni rahatsız eden bir şey mi var', 'sizi ra...","['seni rahatsız eden bir şey mi var oğlum', 's...",NaN,NaN
1004,için yapabileceğim bir şey varsa,272,"['senin için yapabileceğim bir şey varsa', 'si...",['eğer senin için yapabileceğim bir şey varsa'...,['gitmeden önce senin için yapabileceğim bir ş...,['eğer gitmeden önce senin için yapabileceğim ...,['daha rahat etmeniz için yapabileceğim bir şe...


In [ ]:
df_fivegram["sixgram"] = df_fivegram["sixgram"].apply(eval)
df_fivegram["sevengram"] = df_fivegram["sevengram"].apply(eval)
df_fivegram["eightgram"] = df_fivegram["eightgram"].apply(eval)
df_fivegram["ninegram"] = df_fivegram["ninegram"].apply(eval)
df_fivegram["tengram"] = df_fivegram["tengram"].apply(eval)

In [116]:
fivegram = df_fivegram["fivegram"].to_list()
frequency_list = df_fivegram["frequency"].to_list()
sixgram_list = df_fivegram["sixgram"].to_list()
sevengram_list = df_fivegram["sevengram"].to_list()
eightgram_list = df_fivegram["eightgram"].to_list()
ninegram_list = df_fivegram["ninegram"].to_list()
tengram_list = df_fivegram["tengram"].to_list()

In [117]:
firestore_fivegram_data = zip(fivegram,frequency_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [118]:
fivegram_dict = {}
for fivegram,frequency_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fivegram_data:
    dict_var = {f"{fivegram}":{"photo":"","frequency":frequency_list,"sixgram":sixgram_list,
    "sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[""]}}
    fivegram_dict.update(dict_var)   

In [120]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fivegram_data").document(f"details").set(fivegram_dict)

update_time {
  seconds: 1670323436
  nanos: 151622000
}

##### Sixgram

In [121]:
df_sixgram

,sixgram,frequency,sevengram,eightgram,ninegram,tengram
1000,ama önce benim için bir şey,60,"['ama önce benim için bir şey yapman', 'ama ön...","['ama önce benim için bir şey yapman gerek', '...",NaN,NaN
1001,söylemek istediğin bir şey mi vardı,60,['bana söylemek istediğin bir şey mi vardı'],['bana özellikle söylemek istediğin bir şey mi...,NaN,NaN
1002,çöz kalbimi lütfen gel sal beni,60,['gel çöz kalbimi lütfen gel sal beni'],NaN,NaN,NaN
1003,bana şaplak atmana izin vermek hoşuma,60,['bana şaplak atmana izin vermek hoşuma gider'],NaN,NaN,NaN
1004,yürüyüşüne devam et ve hep kendine,60,['kendince yürüyüşüne devam et ve hep kendine'...,['kendince yürüyüşüne devam et ve hep kendine ...,['sadece kendince yürüyüşüne devam et ve hep k...,NaN


In [ ]:
df_sixgram["sevengram"] = df_sixgram["sevengram"].apply(eval)
df_sixgram["eightgram"] = df_sixgram["eightgram"].apply(eval)
df_sixgram["ninegram"] = df_sixgram["ninegram"].apply(eval)
df_sixgram["tengram"] = df_sixgram["tengram"].apply(eval)

In [39]:
sixgram = df_sixgram["sixgram"].to_list()
frequency_list = df_sixgram["frequency"].to_list()
sevengram_list = df_sixgram["sevengram"].to_list()
eightgram_list = df_sixgram["eightgram"].to_list()
ninegram_list = df_sixgram["ninegram"].to_list()
tengram_list = df_sixgram["tengram"].to_list()

In [40]:
firestore_sixgram_data = zip(sixgram,frequency_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [41]:
sixgram_dict = {}
for sixgram,frequency_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fivegram_data:
    dict_var = {f"{sixgram}":{"photo":"","frequency":frequency_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[""]}}
    sixgram_dict.update(dict_var)   

In [42]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sixgram_data").document(f"details").set(sixgram_dict)

##### Sevengram

In [24]:
df_sevengram

,sevengram,frequency,eightgram,ninegram,tengram
1000,en önemli şey daima buradaki insanlar olacak,35,['hayattaki en önemli şey daima buradaki insan...,['ama hayattaki en önemli şey daima buradaki i...,[]
1001,ismi gelen iki pisliği geri götürmeye geldim,35,['masama ismi gelen iki pisliği geri götürmeye...,[],[]
1002,iki adam hakkında sana bir şey diyeyim,35,['bu iki adam hakkında sana bir şey diyeyim'],[],[]
1003,dostum işinizin ehli olduğunuzu biliyor ve saygı,35,['bak dostum işinizin ehli olduğunuzu biliyor ...,['bak dostum işinizin ehli olduğunuzu biliyor ...,[]
1004,şüphe yok ki gemiyi ve bizim hayatlarımızı,35,['hiç şüphe yok ki gemiyi ve bizim hayatlarımı...,['hiç şüphe yok ki gemiyi ve bizim hayatlarımı...,[]


In [ ]:
df_sevengram["eightgram"] = df_sevengram["eightgram"].apply(eval)
df_sevengram["ninegram"] = df_sevengram["ninegram"].apply(eval)
df_sevengram["tengram"] = df_sevengram["tengram"].apply(eval)

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2 Level", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1669369148
  nanos: 63795000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass